# Outline

* [De-trending climate data](#De-trending-climate-data)
  * [Setup](#Setup)
    * [Software](#Software)
    * [Data](#Data)
  * [Initial Exploration](#Initial-Exploration)
  * [Try with real data](#Try-with-real-data)



# De-trending climate data

For some projects, we need to use the projected climate scenarios 
without the warming (or cooling) trend that is present in each 
month (or season).

For instance in a situation where, over 100 years, January
becomes warmer, but July becomes cooler, we would like to remove 
the warming trend from January and the cooling trend from July.

We'd like to remove any trend that might be present for a given month
over the next 100 years, for every pixel in the AIEM domain. 

A single climate driver variable for dvmdostem, over the whole 
AIEM domain for 100 years at 1km spatial resolution and 1 monthly 
time resolution takes ~21GB per variable:

    1850*2560 = 4,736,000 pixels in AIEM domain, including oceans, etc.

    4 bytes
    ----------- * 12 months * 200 years * (1850*2560) = 22,732,800,000 bytes
    32bit float

    22,732,800,000 / 1024 / 1024 / 1024 = 21.17 GB


We need to do this for 4 variables (air temp, vapor pressure, 
near infrared radiation, and precipitation) for six scenarios (echam, 
hadley? ccma? etc?). Thus we need to process ~500GB of data.

In this notebook we will outline the process for removing
a trend from the timeseries climate data downloaded from 
SNAP in a (hopefully) computationally reasonable amount of time: (??)

## Setup

### Software

This notebook will call some external shell commands, so you must 
have the appropriate software installed and available on the path 
that IPython uses. For the most part things should work if you 
install your software with the system package manager (e.g. apt-get 
or yum or homebrew)

* GDAL command line utilities (specifically `gdalbuildvrt`)

Also you will need the following python libraries.

* `netCDF4`
* `matplotlib`
* `scipy.signal`
* `rasterio`
* `IPython Notebook` (for running or adding to this .ipynb file)

### Data

The code in this notebook assumeses that you have downloaded the climate
files from SNAP and have extracted all the `.tifs` so that you have a 
directory strucutre something like this (assuming you are working with 
temperature). Note that the data is organized with a `.tif` image for each
month for the next 100 years (1200 files):

    ├── tas_mean_C_iem_cccma_cgcm3_1_sresa1b_2001_2100
    │   ├── tas_mean_C_iem_cccma_cgcm3_1_sresa1b_01_2001.tif
    │   ├── tas_mean_C_iem_cccma_cgcm3_1_sresa1b_01_2002.tif
    |   ........
    │   ├── tas_mean_C_iem_cccma_cgcm3_1_sresa1b_12_2099.tif
    │   └── tas_mean_C_iem_cccma_cgcm3_1_sresa1b_12_2100.tif

For this notebook, we are working in a directory 


## Initial Exploration

There are many techniques for de-trending data. Ideally we can
use `scipy.signal.detrend` from SciPy's signal processing library
and avoid having to devise our own custom detrending algorithm. 

So first we need to play around with `scipy.signal.detrend` until 
we are convinced it will work for our needs.

Start by loading some libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal

# IPython "magic" to allow plots to be displayed inline
%matplotlib inline

Next, create an x range and a few basic signals over that range:

In [ ]:
SIZE = 1000
x = np.arange(0, SIZE)

noise = np.random.uniform(low=-50, high=50, size=SIZE)
sin_wave = 100 * np.sin( (np.pi/180) * x )
trend = np.linspace(-500, 500, SIZE)

Checkout what we have created so far:

In [ ]:
plt.plot(x, noise, label='noise')
plt.plot(x, sin_wave, label='sin wave')
plt.plot(x, trend, label = 'trend')
plt.legend()

Now we can add our basic components together to create a 
noisy, trending, sin wave. This should be somewhat similar
to how one of our climate variables might look:

In [ ]:
noisy_sin = noise + sin_wave
noisy_trending_sin = noisy_sin + trend

plt.plot(x, noisy_sin, label='noisy sin wave')
plt.plot(x, noisy_trending_sin, label='noisy, trending, sin wave')
plt.legend()

Looking pretty good. Now to see what exactly the 
`scipy.signal.detrend` function does. There are two 
options for how you'd like the detrending to happen, 
'linear' and 'constant'. Accodring to the [documentation](http://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.detrend.html#scipy.signal.detrend),
the linear subtracts the result of a least-squares fit for the
the signal from the signal. The constant option subtracts 
the mean of the signal from the original signal.

In [ ]:
lin_detrend = scipy.signal.detrend(noisy_trending_sin, type='linear')
con_detrend = scipy.signal.detrend(noisy_trending_sin, type='constant')

plt.plot(x, noisy_trending_sin, label='noisy trending sin')
plt.plot(x, lin_detrend, label='linear detrend')
plt.plot(x, scipy.signal.detrend(noisy_trending_sin, type='constant'), label='constant detrend')

plt.legend()

Huh, interesting. Looks like the linear option is what we want
for our application.

However, it looks like the linear detrending seems to center the 
resulting signal around zero, so we will need to offset the 
result to line up with the initial reading in the initial signal:

In [ ]:
offset_lin_detrend = lin_detrend + (noisy_trending_sin[0] - lin_detrend[0])
plt.plot(x, noisy_trending_sin, label='noisy trending sin')
plt.plot(x, offset_lin_detrend,  label='offset linear detrend')
plt.legend()

So that is looking pretty darn good. If we end up needing a 
more complicated line fitting algorithm (e.g. cubic spline) for 
finding the trend maybe we can riff on the `scipy.signal.detrend`.

For now this seems like it should work. Lets try it on some of 
our actual data. But first, lets cleanup and release the memory
needed for this test:

In [ ]:
%reset -f

## Try with real data

First we need to load up some libraries for working with the `.tif`
files from SNAP. After the reset magic from above, we have to grab
numpy, matplotlib and scipy.signal again too:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal

import rasterio
import netCDF4  # Not using this yet, but may want to write netcdf file??
%matplotlib inline

Next we will use GDAL's notion of "Virtual Raster Tiles" 
to assemble a "view"of our 1200 files that shows only one 
month, but for the entire  set of years. We are essentially
creating a new file by combining all the files that would be
listed with this command: 

    $ ls tas_mean_C_iem_cccma_cgcm3_1_sresa1b_01_*.tif
    
So, 100 files; 100 years worth of January.
    
GDAL Utilities has a tool, `gdalbuildvrt`, that can quickly 
create such a "view file", or "Virtual Raster Tile" file that 
behaves just like a `.tif` would. This is essentially an "index"
into a set of `.tif` input files, and allows us to access the data
in the "shape" we want without having to create a copy of the data.
The "Virtual Raster Tile" file has extension `.vrt`. Beacause a 
`.vrt` file behaves identically to a `.tif`, we can open and 
handle the `.vrt` files with the same programs or tools we 
might use on a `tif` file, but the `.vrt` file can be composed 
from a subset of one or more bands in one or more input files. 

In our case we want to be able to open the `.vrt` file and read it 
into a `numpy` datastructure so that we can use `signal.scipy.detrend` 
on the data; we will likely want to write this detrended numpy 
datastructure out to a new file as well.

We will use the exclamation mark to access the system shell commands
for `ls` and `gdalbuildvrt`. We will also use the IPyhton magic to "time"
the execution, and "reset" to free up some memory.

In [ ]:
# Create a list of the files we'd like to include 
# in our virtual dataset. For starters, we'll 
# just pick January for all years:
!ls ../../snap-data/tas_mean_C_iem_cccma_cgcm3_1_sresa1b_2001_2100/tas_mean_C_iem_cccma_cgcm3_1_sresa1b_01* > janfiles.txt

# Next, we ask gdalbuildvrt to create a .vrt file
# from all the files in the list we just created.
#   -seperate      each file to a stacked band
#   -b 1           use band number 1 from each input file
%time !gdalbuildvrt -separate -b 1 -input_file_list janfiles.txt jan.vrt

Not too bad for creating an index into ~20GBs of (sorted) data.

> NOTE: Not sure what the malloc error in `gdalbuildvrt` 
> is all about. Seems to come up after the operation is "done"
> though, so it seems like our file should be ok. Maybe we should
> report a bug.

Now we can open and read the `.vrt` file just as we would 
a `.tif` file:

In [ ]:
# !! NOTE !! uses ~15 seconds, ~1.7GB of RAM!
with rasterio.open('jan.vrt') as src: 
    temperature_data = src.read(masked=None)

I was not able to get Rasterio's masking to work, so I read the 
data in and then we will work on masking out the invalid pixels.

This is for one month, so to do all months, we are looking at 
`(~15secs * 12months)/(60secs/min) ~= 3min`, so about three minutes of 
file-reading time for one variable, one scenario. 
~3 min x 4 variables x 6 scenarios comes out to ~72 minutes 
of file-reading time for all our data. Not trivial, but totally
reasonable for nearly 500GBs of data...

If we look at the data we can see that it looks to be the correct shape.
The axes (dimensions) appear to be (time, y, x). So we have a "stack" of 
images, with each item in the stack being an image along the time axis:

In [ ]:
def print_mask_report(data):
  '''Print some info about a masked array'''
  shape = data.shape
  sz = data.size
  mv = np.count_nonzero(data.mask)
  print(type(mv), type(sz))
  print(mv, sz)
  pcnt = 100.0* mv/sz
  print("Shape: %s Total size: %s Masked values: %s. Percent Masked: %0.7f" % (str(shape), sz, mv, pcnt))

s = np.random.uniform(0,100,1000)
s = np.ma.masked_outside(s, 10, 90)
print_mask_report(s)

In [ ]:
# dimensions are (time, y, x) 
# from upper left corner of image
# (not sure which corner of the pixel)

print("Type of data: ", type(temperature_data))
print("Size of data: ", temperature_data.nbytes/1024/1024, "(Mbs)")
print("Shape of data: ", temperature_data.shape)
print("Min value: ", np.min(temperature_data))
print("Max value: ", np.max(temperature_data))
print("------------------")
print("Using mask?:", temperature_data.mask)
print("Fill value:", temperature_data.fill_value)
print("First value (should be masked):", temperature_data[0,0,0])

Not sure why, but it seems that even though our data is a 
masked array and the fill value seems correct, the mask doesn't
seem to be used because the upper left corner pixel, which should
be masked out, shows up. If it were masked, printing the value
should result in '--', or some other indication that the data
is hidden.

In [ ]:
np.count_nonzero(temperature_data.mask)

If we don't ensure that the oceans and other no-data areas are 
recoginized and appropriately excluded from calculations and displays, 
then the automatic scaleing can make plots difficult to interpert. Usually
the scale ends up far too large in order to accomodate the very-large or
very-small values used for no-data pixels.

There might be a faster way to do this, but this seems to work:

In [ ]:
temperature_data = np.ma.masked_outside(temperature_data[:,:,:], -500, 500, copy=False) 
np.count_nonzero(temperature_data.mask)

In [ ]:
plt.imshow(temperature_data[0])
plt.title("temperature, timestep0")
plt.colorbar()

After going thru the entire detrending and conversion process 
one time, I discovered that there are a few "bad" pixels that have 
data missing somehere along the time axis but are not necessarily
"bad" at all timesteps. These values can results in `RunTime` warnings 
or errors, which I think these are coming from trying to perform math 
on the very large or very small values used for missing data.

If there were no "bad" pixels, then we'd expect the number of masked
pixels to be constant through time (we haven't implemented any "coastal
erosion" features yet!). Instead, it looks like we have a varying number of
masked pixels through time.

In [ ]:
count_masked = [np.count_nonzero(img.mask) for img in (temperature_data)]

mincnt = np.min(count_masked)
maxcnt = np.max(count_masked)
diff = mincnt-maxcnt

print("# of masked values min, max: %i, %i. (delta: %i)" % (mincnt, maxcnt, maxcnt-mincnt))

p = plt.plot(count_masked[:], label="# of masked pixels")
plt.xlabel("timestep")
plt.legend()

What we'd like is one mask that excludes every pixel that has
an undefined value anywhere along its time dimension. Fortunately
`numpy.ma` provides an "any" function that does just that.

To see examples where the pixels are defined in the first timeslice, 
but somewhere along the time axis are undefined, we invert the mask
for the first timeslice and combine it with the "any" mask. Remember,
a `True` value in the mask means "yes this item _is_ to be hidden".

There are not very many of these 'bad' pixels, which makes them very
difficult to actually notice when looking at a map with a normal
colorbar.

In [ ]:
# gives us a 2D array with all "bad" pixels masked.
aggressive_mask = temperature_data.mask.any(0)

# pixels that are good at timestep zero, but masked due to
# a bad value somewhere along the time axis
plt.imshow( np.logical_and(aggressive_mask, np.invert(temperature_data[0].mask)), cmap='gray', interpolation='none')
plt.colorbar()

We can look more closely at this area to make sure we
know what is going on.

In [ ]:
T0 = 0
T1 = 95

fig, ((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3)

# large zoom, bristol bay
yl=1200; yh=1700; xl=400; xh=900; extents=[xl,xh,yh,yl]
ax1.imshow( temperature_data[T0,yl:yh,xl:xh], interpolation='none', extent=extents )
ax1.set_title("large zoom t=%s"%T0)

# med zoom, bristol bay
yl=1250; yh=1400; xl=550; xh=700; extents=[xl,xh,yh,yl] 
ax2.imshow( temperature_data[T0,yl:yh,xl:xh], interpolation='none', extent=extents )
ax2.set_title("med zoom t=%s"%T0)

# Tiny zoom bristol bay
yl=1300; yh=1310; xl=640; xh=650; extents=[xl,xh, yh, yl]
ax3.imshow( temperature_data[T0,yl:yh,xl:xh], interpolation='none', extent=extents )
ax3.set_title("tight zoom t=%s"%T0)
plt.tight_layout()

# large zoom, bristol bay
yl=1200; yh=1700; xl=400; xh=900; extents=[xl,xh,yh,yl]
ax4.imshow( temperature_data[T1,yl:yh,xl:xh], interpolation='none', extent=extents )
ax4.set_title("large zoom t=%s"%T1)

# med zoom, bristol bay
yl=1250; yh=1400; xl=550; xh=700; extents=[xl,xh,yh,yl] 
ax5.imshow( temperature_data[T1,yl:yh,xl:xh], interpolation='none', extent=extents )
ax5.set_title("med zoom t=%s"%T1)

# Tiny zoom bristol bay
yl=1300; yh=1310; xl=640; xh=650; extents=[xl,xh, yh, yl]
ax6.imshow( temperature_data[T1,yl:yh,xl:xh], interpolation='none', extent=extents )
ax6.set_title("tight zoom t=%s"%T1)

In [ ]:
plt.plot(temperature_data[:,1301,642])

In [ ]:
plt.plot(scipy.signal.detrend(temperature_data[:,1301,642]) )
plt.plot(scipy.signal.detrend(temperature_data[:,1305,642]) )

So now what we want to do is take the "any" mask, (most
aggressive, hides all pixels that don't have complete time-
series), and apply that to each image in our series.
                                            , 

In [ ]:
# MAYBE DON'T EVEN NEED THIS?
# since detrend seems to ignore the mask anyways...
# Apply the most aggressive mask to every timeslice.
# for i, img in enumerate(temperature_data[:]):
#     temperature_data.mask[i,:,:] = temperature_data.mask.any(0)

In [ ]:
dtrd = scipy.signal.detrend(temperature_data, axis=0)

In [ ]:
# THIS PRODUCES OVERFLOW...
#np.float32(3.4028235e+38) + np.float32(3.4028235e+38)

dtrd = np.ma.masked_outside(dtrd, -100, 100)

count_masked_d = [np.count_nonzero(img.mask) for img in dtrd]

mincnt_d = np.min(count_masked_d)
maxcnt_d = np.max(count_masked_d)
diff_d = mincnt_d - maxcnt_d

print("ORIG # of masked values min, max: %i, %i. (delta: %i)" % (mincnt, maxcnt, maxcnt - mincnt))
print("DTRD # of masked values min, max: %i, %i. (delta: %i)" % (mincnt_d, maxcnt_d, maxcnt_d - mincnt_d))
p = plt.plot(count_masked[:], label="# of masked px odata")
p = plt.plot(count_masked_d[:], label="# of masked px dtrddata")
plt.xlabel("timestep")
plt.legend()

In [ ]:
plt.imshow(dtrd[0,500:1500,0:1500])
#plt.imshow(np.ma.masked_outside(dtrd[2,:,:], -100, 100))

In [ ]:
# Now write each timestep to a nc file?

In [ ]:
plt.plot(dtrd[:,1301,642])

In [ ]:
dtrd_ofs = dtrd + (temperature_data[0,:,:] - dtrd[0,:,:])

In [ ]:
plt.imshow(dtrd_ofs[0,1250:1350,550:650], interpolation='none')

In [ ]:
a = np.arange(0, 1000).reshape((10,10,10))
a[0,3:7,3:7] = -99999
a = np.ma.masked_outside(a, -6000,6000)
#plt.imshow(a[0], interpolation='none')
b = a + 100
# plt.imshow(b[0], interpolation='none')
# plt.colorbar()

# plt.imshow( (a[0]**2+a[0:4])[0] )
# plt.colorbar()
# a = np.ma.masked_inside(a, 30, 70)
# plt.plot(a)
# b = a + np.arange(0,100)
# plt.plot(b)


sdt = scipy.signal.detrend(a, axis=0)
plt.plot(sdt[:,1,1])
plt.plot(a[:,1,1])

In [ ]:
# Next, we can look at the timeseries for a 
# pixel in central alaska that should have data
plt.plot(temperature_data[:, 550, 1050])

Now we can run the `scipy.signal.detrend` function over
the time axis for every pixel on the map:

In [ ]:
# Another 15 seconds or so and a lot of memory. There might
# be a better way to do this.
temperature_data.mask[0] = temperature_data.mask.any(0)
temperature_data.mask.shape

%time dtrd = scipy.signal.detrend(temperature_data, axis=0)

And then look at the detrended timeseries for
a pixel. The `scipy.signal.detrend` centers the resulting
signal around zero. We can offset the signal back to its 
original position by adding the first value of the original
timeseries to the detrended vector.

In [ ]:
Y = 550
X = 1050
plt.suptitle("Pixel(y,x): (%s, %s)" % (Y, X))
plt.plot(temperature_data[:,Y,X], label='original')
plt.plot(dtrd[:,Y,X], label='detrended')
plt.plot((dtrd[:,Y,X] + (temperature_data[0,Y,X] - dtrd[0,Y,X])), label='dtrend+offset')
plt.legend()

It turns out it is blazingly fast to add the offset
to the array in place:

In [ ]:
%time dtrd[:,:,:] += (temperature_data[0,:,:] - dtrd[0,:,:])

Now the timeseries for each pixel in the image
has been detrended and offset so that the detrended
timeseries begins at the same value as the original 
timeseries.

In [ ]:
dtrd = np.ma.masked_outside(dtrd[:,:,:], -500, 500, copy=False)

In [ ]:
print(temperature_data.min(), temperature_data.max())
print(dtrd.min(), dtrd.max())

In [ ]:
def check_pixel(orig, detrended, Y = 550, X = 1050):
    plt.suptitle("Pixel(y,x): (%s, %s)" % (Y, X))
    plt.plot(orig[:,Y,X], label='original')
    plt.plot(detrended[:,Y,X], label='detrended')
    plt.legend()

In [ ]:
check_pixel(temperature_data, dtrd, Y=550,X=1050)

In [ ]:
check_pixel(temperature_data, dtrd, Y=1200, X=780)

Now that we have the method worked out we will write a
more generalized script that can be used on all four variables
and likely will concatenate the months to create new output series
in the same shape/format as the original input tifs.


In [ ]:
# Yikes, lots of memory use! 
# See what we are using:
print(dtrd.nbytes/1024.0/1024.0/1024.0, "GBs")
print(temperature_data.nbytes/1024.0/1024.0/1024.0, "GBs")

In [ ]:
check_pixel(temperature_data, dtrd, Y=992, X=321)

In [ ]:
type(temperature_data[0,500,500])

In [ ]:
plt.imshow(temperature_data[0,:,:].mask)

In [ ]:
#plt.imshow(temperature_data[0,:,:].mask - temperature_data[:,:,:].mask.any(0))
plt.imshow(np.logical_or(temperature_data[:,:,:].mask.any(0), temperature_data[0,:,:].mask))

In [ ]:
# Larget area, has some issues...
# T=7
# yl = 1245
# yh = 1755
# xl = 287
# xh = 797
# extents=[xl,xh, yh, yl]

# med tight zoom, bristol bay
# T=7
# yl = 1250
# yh = 1455
# xl = 550
# xh = 700
# extents=[xl,xh, yh, yl]

# Tiny zoom bristol bay
T=90
yl = 1300
yh = 1310
xl = 640
xh = 650
extents=[xl,xh, yh, yl]


# SUPER TIGHT ZOOM IN ON 
# island sw of Kodiak. Has some issues, first timestep...
# T=0
# yl = 1645
# yh = 1655
# xl = 687
# xh = 697
# extents=[xl,xh, yh, yl]

otemps = temperature_data[T,yl:yh,xl:xh]
dtemps = dtrd[T,yl:yh,xl:xh]
mdtemps = np.ma.masked_outside(dtemps, -100, 100)


fig, (ax1, ax2) = plt.subplots(1,2)

ax1.imshow(otemps, extent=extents, interpolation='none')
ax1.set_title("original temps t=%s"%T)

ax2.imshow( mdtemps, interpolation='none', extent=extents)
ax2.set_title("detrended temps t=%s"%T)


print(np.min(otemps), np.max(otemps), (np.min(otemps) < 0 and (np.min(otemps) > -1))) # and 
print(np.min(dtemps), np.max(dtemps))
print(np.min(mdtemps), np.max(mdtemps))

# print "Original Temps:", otemps
# print "Detrended Temps:", dtemps
# print "Maseked, Detrended Temps:", mdtemps



#plt.colorbar(temperature_data[T,yl:yh,xl:xh])
# extent=[80,120,32,0]

#ax1.imshow(temperature_data[0:1250,1750:250,750])

In [ ]:
check_pixel(temperature_data, dtrd, X=642, Y=1301)
plt.legend(loc='lower left')

In [ ]:
3.179043990183396e-37 <0

In [ ]:
type(temperature_data)

In [ ]:
a = np.zeros((5,3,3))
a[4,2,1] = np.inf
am = np.ma.masked_outside(a, -100, 100)
print(am.mask)
am.mask.any(0)

In [ ]:
#anything masked along the zero-th dimension, but not masked in the first timeslice, so "false".
plt.imshow( np.logical_and(temperature_data.mask.any(0), np.invert(temperature_data[0].mask)), cmap='gray')
plt.title("These pixels are missing data somewhere along the time axis!!, but are defined in the first timeslice")

In [ ]:
a = np.linspace(0,99, 100).reshape((10,10))
b = np.ma.masked_outside(a, 30, 80).mask
c = np.ma.masked_inside(a, 50, 60).mask
d = np.logical_or(b,c)
e = np.logical_and(c, ~b)
# print a
# print b
# print c
# print d
# print e


fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1,5)

ax1.imshow(a, interpolation='none', cmap='jet')
ax1.set_title("a")

ax2.imshow( b, interpolation='none', cmap='gray', alpha=1.0)

ax2.set_title("b")

ax3.imshow( c, interpolation='none', cmap='gray', alpha=1.0)
ax3.set_title("c")

ax4.imshow( d, interpolation='none', cmap='gray', alpha=1.0)
ax4.set_title("d=(c||b)")

ax5.imshow( np.invert(e), interpolation='none', cmap='gray', alpha=1.0)
ax5.set_title("")

In [ ]:
a[1,1] = 3000
a_mask = np.ma.masked_outside(a, -100, 100).mask

In [ ]:
a_mask

In [ ]:
%reset -f

In [ ]:
%reset -f